# Tensorflow 的一个简易的卷积神经网络

在上一篇notebook中，我们通过剖析一个简单的logistic regression分类器，说明了用Tensorflow搭建模型的基本步骤。今天，我们继续使用MNIST数据集，来进一步搭建一个简单的卷积神经网络，毕竟Tensorflow的最大威力在于其在深度模型中的使用。这里我们默认读者有基本的卷积神经网络知识。

### 读取MNIST数据
如果有对MNIST不了解的读者可以参考其[官网](http://yann.lecun.com/exdb/mnist/)。我们在这里调用一份现成的代码来读取这个数据集。首先[在这里](https://tensorflow.googlesource.com/tensorflow/+/master/tensorflow/examples/tutorials/mnist/input_data.py)下载读取代码，然后用下面的code将MNIST数据集读进Python

In [1]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Convolutional Neural Network
这里，我们使用相对于Seesion更加方便的InteractiveSession。通过它，你可以更加灵活地构建你的代码。它能让你在运行的时候，插入一些Graph。这对于工作在交互式环境中的人来说非常便利，比如使用IPython。如果你不使用InteractiveSession，那么你需要在启动session之前构建整个Graph，然后启动该Graph。

为了卷积神经网络，我们需要把x变成一个4d向量，其第2、第3维对应图片的宽、高，最后一维代表图片的颜色通道数(因为是灰度图所以这里的通道数为1，如果是rgb彩色图，则为3)。

In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()

# define x and y_
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

x_image = tf.reshape(x, [-1,28,28,1])

这里，我们使用的是ReLU激活函数，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为零的问题（dead neurons）。为了不在建立模型的时候反复做初始化操作，我们定义两个函数用于初始化。

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

关于卷积的选择，我们使用5乘5的步长为1，有padding的filter，来保证输出和输入是同一个大小。pooling用简单的2乘2大小的max pooling。

In [6]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

定义第一个卷积层的参数

In [7]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

计算第一个卷积层

In [8]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

定义和计算第二个卷积层

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

全连接层的定义和计算，注意7是28经过两次2乘2的pooling之后得到的。所以全连接之前我们一个有64的7乘以7的filter。一共有7乘7乘64个feature

In [10]:
# 7 = 28/2/2
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# reshape the h_pool2 for the fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

这里我们用dropout来避免overfitting。我们用一个placeholder来代表dropout的概率。注意我们要在训练过程中启用dropout，在测试过程中关闭dropout。 TensorFlow的tf.nn.dropout操作除了可以实现dropout以外，还会自动处理神经元输出值的scale。所以用dropout的时候可以不用考虑scale。

In [11]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

预测层的定义和计算

In [12]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

和上一篇notebook一样，我们定义好模型之后，接下来我们就可以选择我们的loss函数和优化算法。这里我们继续使用cross-entropy作为loss函数，使用Adam作为优化算法，Adam可以说是目前最常用的优化算法之一。同样，我们继续使用accuracy作为衡量模型的指标。

In [13]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

在开始真正训练模型之前，别忘了初始化所有的variable。这里，我们选用 50 mini batch gradient descent 训练模型5000遍， 并每隔500次输出一个accuracy。最后输出testing data的accuracy。

In [14]:
sess.run(tf.initialize_all_variables())

for i in range(5000):
    
    batch = mnist.train.next_batch(50)
    
    if i%500 == 0:   
        # notice accuracy.eval; predicting using keep prob 1.0
        train_accuracy = accuracy.eval(feed_dict={x : batch[0], y_: batch[1], keep_prob: 1.0})   
        print("step %d, training accuracy %g"%(i, train_accuracy))
    
    # notice train_step.run; trainning using keep prob 0.5
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# testing data accuracy    
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


step 0, training accuracy 0.04
step 500, training accuracy 0.88
step 1000, training accuracy 0.98
step 1500, training accuracy 0.98
step 2000, training accuracy 0.96
step 2500, training accuracy 0.98
step 3000, training accuracy 1
step 3500, training accuracy 0.98
step 4000, training accuracy 1
step 4500, training accuracy 0.98
test accuracy 0.9868


这篇notebook为大家一步一步构建了一个简单的卷积神经网络。虽然不是很复杂，但是包含了一个卷积神经网络应有的全部元素。

UniDeep 数据科学家将为大家定期奉上Tensorflow和Keras的学习心得。我们会与大家一起从安装Tensorflow开始，系统地整理Tensorflow的学习笔记，并为大家整理Tensorflow在一些经典深度学习模型， 如：AlexNet， ResNet， LSTM上实现的代码。最后我们也会为大家介绍Keras， 这一基于Tensorflow框架的，简单方便的神经网络构建模块。

接下来，我们将为大家展示Tensorflow的一些高级feature。敬请关注！